In [1]:
import pandas as pd

df = pd.read_csv('../all_ranks.csv')
df.fillna(0, inplace=True)
data = df.to_dict('records')
df

,RankId,Rank Name,RequirementId,Requirement Name,Requirement Short,Requirement Number,Required
0,1,Scout,2007,"Repeat from memory the Scout Oath, Scout Law, ...",Scout Oath & Law,1a,True
1,1,Scout,2009,Explain what Scout spirits is. Describe somewa...,Scout Spirit,1b,True
2,1,Scout,2010,"Demonstrate the Scout sign, salute, and handsh...",Salute & Sign,1c,True
3,1,Scout,2011,Describe the First Class Scout badge and tell ...,First Class Badge,1d,True
4,1,Scout,2012,Repeat from memory the Outdoor Code. List the ...,Outdoor Code,1e,True
...,...,...,...,...,...,...,...
138,7,Eagle Scout,1510,Earn a total of 21 merit badges (10 more than ...,Earn 21 merit badges,3,True
139,7,Eagle Scout,1511,"While a Life Scout, serve actively in your tro...",Position of responsibility,4,True
140,7,Eagle Scout,1512,"While a Life Scout, plan, develop, and give le...",Eagle project,5,True
141,7,Eagle Scout,1513,"While a Life Scout, participate in a Scoutmast...",Scoutmaster conference,6,True


In [2]:
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('all-mpnet-base-v2')

qdrant = QdrantClient(":memory:")

def create_collection(collection_name):
    return qdrant.create_collection("all_ranks",
                               vectors_config=models.VectorParams(
                                   size=encoder.get_sentence_embedding_dimension(),
                                   distance=models.Distance.COSINE
                                   )
                                   )

if qdrant.collection_exists("all_ranks"):
    qdrant.delete_collection("all_ranks")
    collection = create_collection("all_ranks")
else:
    collection = create_collection("all_ranks")


In [3]:
qdrant.upload_points(
    collection_name="all_ranks",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Requirement Name"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
    )

In [5]:
search_results = qdrant.search(
    collection_name="all_ranks",
    query_vector=encoder.encode("fire").tolist(),
    limit=5
)

In [37]:
prompt = "Describe activities that would meet BSA rank requirements that have to do with fire safety and include a list of materials needed for each activity."

# Adapted from OpenAI's Vision example 
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/deepseek-llm-7B-base-GGUF",
  messages=[
    {
      "role": "system",
      "content": "You are mentoring a Scoutmaster. As a mentor, assist the user with their questions regarding the ScoutsBSA program.",
    },
    {
      "role": "user",
      "content": [
          {"type": "text", "text": prompt},
          {"type": "text", "text": "These are the most likely rank requirements that have to do with fire safety:" + "\n".join([search_result.payload["Requirement Name"] for search_result in search_results])}
      ]
    }
  ],
  max_tokens=5000,
)


In [38]:
completion.choices[0].message.content

'Describe activities that would meet BSA rank requirements that have to do with fire safety and include a list of materials needed for each activity.<｜end▁of▁sentence｜><｜begin▁of▁sentence｜>ScoutsBSA Requirements 2019\nBoy Scouts Of America\n# 5a - Health And Safety\n## Learning Outcomes\nThe Scout learns the importance of personal health, safety, and protection from disease by developing and maintaining good habits related to nutrition, hygiene, exercise, water quality, injury prevention, and first aid. The Scout is also educated about the diseases that can be transmitted through blood.\n'